# Using Vector Search in Weaviate and Azure Open AI

In [20]:
# Import libraries
import os
from dotenv import load_dotenv
import json
import requests
import weaviate
from weaviate import EmbeddedOptions
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import MetadataQuery, HybridVector, Move
import warnings

warnings.filterwarnings('ignore')

# Load the environment variables
load_dotenv()

aoai_key=os.environ['AZURE_OPENAI_API_KEY']
aoai_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
aoai_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
aoai_embedding=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
aoai_embedding_3=os.environ['AZURE_OPENAI_EMBEDDINGS_3_DEPLOYMENT']

## Utility Funtions

In [22]:
# JSON print beautifier
def json_print(data):
    print(json.dumps(data, indent=2))

## Create or connecto to a Weaviate Embedded DB

* https://weaviate.io/developers/weaviate/connections/connect-embedded
* https://weaviate.io/developers/weaviate/connections/connect-local

In [ ]:
try:
    response = requests.get("http://localhost:8079/v1/schema")
    r_err = False
except Exception as err:    
    r_err = True

if r_err:
    client = weaviate.connect_to_embedded(
        version="1.26.1",  # e.g. version="1.26.5"
        headers={
            "X-OpenAI-BaseURL": aoai_endpoint,
            "X-Azure-Api-Key": aoai_key
        },
    )
    print("Connected to new instance")
elif response.status_code == 200:
    client = weaviate.connect_to_local(
        port=8079, 
        grpc_port=50050,    
        headers={
            "X-OpenAI-BaseURL": aoai_endpoint,
            "X-Azure-Api-Key": aoai_key
        }
    )
    print("Connected to existing instance")

print(client.is_ready())

In [ ]:
# Show Weaviate DB metadata
json_print(client.get_meta())

## Create a new Collection (EU Destinations)

https://weaviate.io/developers/weaviate/manage-data/collections

In [25]:
if client.collections.exists("eudestinations"):
    client.collections.delete("eudestinations")

In [ ]:
# Using a Weaviate collection with text-embedding-ada-002
client.collections.create(
    "eudestinations",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="title_vector",            
            resource_name="aoai-airlift-1",
            deployment_id=aoai_embedding,
            base_url=aoai_endpoint
        )
    ]
)

## Read Europe Touristic Destinations Dataset

_Source: https://www.kaggle.com/datasets/faizadani/european-tour-destinations-dataset_

In [ ]:
file_path = '../data/eu_destinations_n.json'

with open(file_path, "r") as file:
    data = file.read()

ds = json.loads(data)
json_print(ds[0])

## Loading Embeddings into the Weaviate DB

https://weaviate.io/developers/weaviate/manage-data/import

In [ ]:
collection = client.collections.get("eudestinations")

with collection.batch.dynamic() as batch:
    for i, d in enumerate(ds):
        weaviate_obj = {
            "destination": d["Destination"],
            "region": d["Region"],
            "country": d["Country"],
            "category": d["Category"],
            "annualtourists": d["Approximate Annual Tourists"],
            "foods": d["Famous Foods"],
            "language": d["Language"],
            "besttimevisit": d["Best Time to Visit"],
            "costliving": d["Cost of Living"],
            "cultural": d["Cultural Significance"],
            "description": d["Description"]
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )

## Query a specific Collection Dataset

In [30]:
collection = client.collections.get("eudestinations")

## Search Patterns

https://weaviate.io/developers/weaviate/search/basics

In [ ]:
# Fetch a DB object in ascending UUID order
response = collection.query.fetch_objects(
    limit=1
)
for o in response.objects:
    json_print(o.properties)

### Sparse and Dense Search

#### Sparse
* BM25 search is one implementation of what is commonly called a 'keyword' search. Broadly speaking, it works by matching the search terms between the query and the data objects in the index.
* The higher the score, the greater the relevance of the object.
* https://weaviate.io/developers/weaviate/search/bm25

In [ ]:
response = collection.query.bm25(
    query="sea",
    return_properties=['country','foods','cultural','description'],
    return_metadata=MetadataQuery(score=True),
    limit=5
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.score}")
    json_print(o.properties)

#### Dense
* A method that uses neural network embeddings to represent and retrieve information based on semantic similarity.
* **Distance closer to 0:** _identical vectors_ / **Distance closer to 2:** _Opposing vectors_.
* https://weaviate.io/developers/weaviate/search/similarity

In [ ]:
response = collection.query.near_text(
    query="sea",  # The model provider integration will automatically vectorize the query
    return_properties=['country','foods','cultural','description'],        
    return_metadata=MetadataQuery(distance=True),
    limit=10
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.distance}")
    json_print(o.properties)

In [ ]:
response = collection.query.near_text(
    query="sea",  # The model provider integration will automatically vectorize the query    
    return_properties=['country','foods','cultural','description'],
    distance=0.219, 
    include_vector=True,
    return_metadata=MetadataQuery(distance=True)
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.distance} \nVector: {o.vector}")
    json_print(o.properties)    

### Hybrid Search

* Hybrid search combines the results of a vector search (Dense search) and a keyword (BM25F) search by fusing the two result sets.
* The higher the score, the greater the relevance of the object.
* https://weaviate.io/developers/weaviate/search/hybrid

In [ ]:
response = collection.query.hybrid(
    query="sea",
    return_properties=['country','foods','cultural','description'],
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=6
    )

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.score} \nExplanation: {o.metadata.explain_score}")    
    json_print(o.properties)   

* Hybrid search results can favor the keyword component or the vector component. To change the relative weights of the keyword and vector components, set the alpha value in your query:
* An alpha of **1** is a pure _vector(dense)_ search / An alpha of **0** is a pure _keyword(sparse)_ search.

In [ ]:
response = collection.query.hybrid(
    query="sea",
    return_properties=['country','foods','cultural','description'],
    alpha=0.7,
    limit=10
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1}")    
    json_print(o.properties) 

* You can specify **vector similarity search parameters**, such as **group by** and **move to** or **move away**. 
* https://weaviate.io/developers/weaviate/api/graphql/search-operators

In [ ]:
response = collection.query.hybrid(
    query="sea",
    return_properties=['country','foods','cultural','description'],
    #max_vector_distance=0.4,  # Maximum threshold for the vector search component
    vector=HybridVector.near_text(
        query="Scenic view",
        move_away=Move(force=0.5, concepts=["Mediterranean"]),
    ),
    # alpha=0.75,
    # limit=10
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1}")    
    json_print(o.properties) 